<a href="https://colab.research.google.com/github/DCharles01/Jigsaw-Colabs/blob/main/6-docker-compose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flask Docker Compose

### Introduction

In this lesson, we'll work through setting up a multi-service application with docker compose.  Our application should consist of a streamlit front-end that hits a flask API in the backend.  That flask api will be able to make requests to a postgres service.

Ok, let's get started.

### Structuring our application

To begin, we can create some folders to place our different services.

```
services/
├── docker-compose.yml
└── api
    |_src
└── frontend
```

### Beginning with the API

Ok, now let's get started with building out a flask api.  Develop an api so that when we visit `/locations`, we see a list of locations that Anthony Bourdain visited in his TV show *No Reservations*.

We can begin by returning the list of locations provided here:

In [ ]:
locations = [{'country': 'France', 'city': 'Paris',
              'coordinates': '48.856614,2.352222'},
               {'country': 'Iceland', 'city': 'Reykjavík',
                'coordinates': '64.126521,-21.817439'}]

Make sure that the environmental `FLASK_APP` points to the appropriate file.

> Use a `.env` file to store the environmental variables.

> Use a `requirements.txt` file to install the flask library.

When it's complete, we should be able to call `flask run` and make a request to `/locations` to see a list of locations.

In [ ]:
import requests
response = requests.get('http://127.0.0.1:5000/locations')
response.json()

# [{'city': 'Paris', 'coordinates': '48.856614,2.352222', 'country': 'France'},
#  {'city': 'Reykjavík',
#   'coordinates': '64.126521,-21.817439',
#   'country': 'Iceland'}]


[{'city': 'Paris', 'coordinates': '48.856614,2.352222', 'country': 'France'},
 {'city': 'Reykjavík',
  'coordinates': '64.126521,-21.817439',
  'country': 'Iceland'}]

Ok, now that we have this initial api built out, let's dockerize our API. This involves a Dockerfile located inside of the `/api` folder.

Here we'll get you started.

```Dockerfile
FROM python:3.7-alpine
WORKDIR /usr/src/app

...

CMD ["flask", "run", "--host=0.0.0.0"]
```

> Remember that we need to set the host argument to expose our server outside of the docker container.

Ok, now build the image `docker build`.

Then run the image so that we can make the following request.

In [ ]:
import requests
response = requests.get('http://0.0.0.0:5000/locations')
response.json()

[{'city': 'Paris', 'coordinates': '48.856614,2.352222', 'country': 'France'},
 {'city': 'Reykjavík',
  'coordinates': '64.126521,-21.817439',
  'country': 'Iceland'}]

### Moving to Docker Compose

Ok, now let's integrate our api with docker-compose file.

Specify top-level keys of `version` and `services`, then define a service called `api`.  For api define both how to build the service and the ports.

> You do not need to specify the environment as this is read from our .env file which is located in the image.

```yaml
version: '3.7'
services:
```

Once you have the docker-compose file for this service started.  Let's try to boot it up.

`docker-compose up`

<img src="https://github.com/jigsawlabs-student/multiservices-compose-lab/blob/main/services-api.png?raw=1" width="70%">

Once again, we should be able to reach `0.0.0.0:5000` and see the list of locations.

### Connecting the Services

Next up is to build an initial streamlit app.  To do this, build out the following structure:
```
frontend
├── Dockerfile
├── requirements.txt
└── src
    └── index.py
```

Now the `requirements.txt` should have the following libraries listed:

```
streamlit
black
altair
```

And in the `index.py` file we can place the following:

```python
import streamlit as st

st.write("""
# My first app
Hello *world!*
""")
```

At this point, we should be able to boot up our streamlit app by installing the libraries specified in `requirements.txt`. And then running `streamlit run src/index.py`.

* Dockerizing streamlit

From here, we can dockerize our streamlit app.  We'll do so by filling in the corresponding `Dockerfile`.  The one trick is that the Dockerfile should inherit from with `python:3.7` instead of `python:3.7-alpine`.

So the first two lines will look like the following:

```Dockerfile
FROM python:3.7
WORKDIR /usr/src/app
```

> We'll let you finish it up the Dockerfile from there.

When it's complete, build the image and then run a container, exposing the appropriate port.

<img src="https://github.com/jigsawlabs-student/multiservices-compose-lab/blob/main/streamlit-initial.png?raw=1" width="60%">

* Integrating with docker compose

Next up is to add the corresponding lines to our docker compose file so that running `docker-compose up` boots up both our api and our frontend.  Add a new service to the docker-compose file called `frontend`, along with rest of the configuration for the service.

### Connecting the two

Now we would like our frontend to use our api, well as an api, and make a request to it.  

1. Bind Mapping

Before getting on with it, it would be nice if we could update our application without rebuilding the image.  We can accomplish this through a bindmapping.  

In the `docker-compose` file, add the following under the `frontend` service.

```yaml
volumes:
      - ./frontend/src:/usr/src/app/src
```

This will map our ports from `src` to the directory in our frontend image.  

> **Notice**: That our source is specified from a path relative to the docker-compose file, but our target is specified using the *absolute path* of the image.

So run `docker-compose up` again, and see if we can make changes to the streamlit app without rebuilding the image.

2. Making requests to the api

Ok, now it's time to make requests to the api.  In the streamlit app, import the requests library and make a request to the locations route in our api.

`api:5000/locations`

Display the json on `streamlit`.

You can do this by writing the json:

```python
locations = response.json()
for location in locations['locations']:
    st.write(location[1])
```

3. Bind mapping again

Then let's bind map the contents of the `/api/src` locally to that in the api container.  This way if we make changes locally, they will immediately be updated in the container.

> To see the absolute path in the container, you may need to sh into the container.

Now one thing is that the container will not *display* live updates made via the api.  This is because our flask app does not have `debug = True`.  That's ok, we can still make a change to say the `__init__` file, then `sh` into the container and `cat` the `__init__` to confirm that the file changed.

### Integrating a Database

Ok, now that we have data coming from our api coming to the streamlit app.  Let's change how our api gets it's data.  Currently, it's just hardcoded into the locations route.

```python
@app.route('/locations')
def locations():
    locations = [{'country': 'France', 'city': 'Paris',
              'coordinates': '48.856614,2.352222'},
               {'country': 'Iceland', 'city': 'Reykjavík',
                'coordinates': '64.126521,-21.817439'}]
    return jsonify(locations)
```

Let's change this so that we get the data from a database.

1. Adding a DB service

First, in the docker-compose file, let's add a db service.  
> We don't need to worry about connecting it to our flask-api just yet.

Ok. let's get started.

This service should use the `postgres` image.  And to get it working we need to do the following:

1. Specify the environment variables `POSTGRES_USER` and `POSTGRES_PASSWORD`
2. Map the port from `5432` to `5432`
3. Specify and create a volume to store data which is stored in `/var/lib/postgresql/data`.

> Try not to refer to the previous lessons in setting this up.  It is ok, to refer to the [postgres documentation](https://hub.docker.com/_/postgres).

Then, let's run docker-compose up.  We should now see the db service running alongside the two other services.

<img src="https://github.com/jigsawlabs-student/multiservices-compose-lab/blob/main/multiple-services.png?raw=1" width="80%">

2. Connecting to api to our db

Next up is to install the postgres package in our api, and connect it to our db service.  First, let's install the appropriate dependencies needed for the postgres package.  To install these dependencies, add the following lines to the api's Dockerfile.

```Dockerfile
RUN apk update \
    && apk add postgresql-dev gcc python3-dev musl-dev
```

Then add the postgres library to the `requirements.txt` file.

And then in the `__init__.py` file, let's setup postgres to connect to our db.

```python
POSTGRES_DATABASE_URI = os.environ.get('DATABASE_URL')
connection = Postgres(POSTGRES_DATABASE_URI)

connection.run("CREATE TABLE locations (name text)")
connection.run("INSERT INTO locations VALUES ('maggie simpson')")
```

And then in the routes we can select from the database and return the data as json.

```python
@app.route('/locations')
def index():
    users = connection.all('SELECT * FROM locations;')
    return jsonify({'locations': locations})
```

3. Refactor the initial database call

Now instead of creating a table each time we boot up our application, we can specify an initialization function directly in sql.  We can do this by adding a volume that establishes a bind mount between the folder with our sql file `/docker-entrypoint-initdb.d/`.

Here, we have an initial set of sql defined in the `db` folder.

### Connect to frontend to the API

Now that we have connected the api to the db, it's time to connect the frontend to the api.  We can do so by making a request to our api from our streamlit application.

Use the requests library to make the correct request.  And use this request to get the data to display the list of cities.

> One gotcha is that you will likely need to start up the db service before running `docker-compose up` to properly make the request.

> <img src="https://github.com/jigsawlabs-student/multiservices-compose-lab/blob/main/app-display.png?raw=1" width="60%">

Ok, with this we have used docker-compose to connect all three of our services.

### Resources

[Docker compose env](https://medium.com/better-programming/using-variables-in-docker-compose-265a604c2006)

[Parts unknown travel places](https://github.com/underthecurve/bourdain-travel-places)

[Return Json from DB](https://varun-verma.medium.com/use-psycopg2-to-return-dictionary-like-values-key-value-pairs-4d3047d8de1b)

[Dockerizing Flask and Unicorn](https://testdriven.io/blog/dockerizing-flask-with-postgres-gunicorn-and-nginx/)

[Docker build](https://stackoverflow.com/questions/46711990/error-pg-config-executable-not-found-when-installing-psycopg2-on-alpine-in-docker)

[Nginx Flask Docker Postgres](http://www.ameyalokare.com/docker/2017/09/20/nginx-flask-postgres-docker-compose.html)

[docker with volumes](https://www.saltycrane.com/blog/2019/01/how-run-postgresql-docker-mac-local-development/)